1. 라이브러리 임포트 및 GPU 설정

In [1]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

2. 데이터 로드 및 전처리

In [12]:
dataset = os.listdir('./new_dataset')
dataset

['00_flower.npy',
 '01_crown.npy',
 '02_heart_beat.npy',
 '03_firework.npy',
 '04_bear.npy',
 '05_cat.npy',
 '06_son_celebration.npy',
 '07_heart_on_the_cheek.npy',
 '08_gun.npy',
 '09_pipe.npy',
 '10_tiger.npy',
 '11_landmarks.npy']

In [15]:
f'./new_dataset/{dataset[11]}'

'./new_dataset/11_landmarks.npy'

In [22]:
actions = [
    'flower', 'crown', 'heart_beat',
    'firework', 'bear', 'cat',
    'son_celebration', 'heart_ont_the_cheek', 'gun',
    'pipe', 'tiger', 'landmarks'
]

data = np.concatenate([
    np.load(f'./new_dataset/{dataset[0]}'),
    np.load(f'./new_dataset/{dataset[1]}'),
    np.load(f'./new_dataset/{dataset[2]}'),
    np.load(f'./new_dataset/{dataset[3]}'),
    np.load(f'./new_dataset/{dataset[4]}'),
    np.load(f'./new_dataset/{dataset[5]}'),
    np.load(f'./new_dataset/{dataset[6]}'),
    np.load(f'./new_dataset/{dataset[7]}'),
    np.load(f'./new_dataset/{dataset[8]}'),
    np.load(f'./new_dataset/{dataset[9]}'),
    np.load(f'./new_dataset/{dataset[10]}'),
    np.load(f'./new_dataset/{dataset[11]}'),
], axis=0)

print(f"Concatenated data shape: {data.shape}")

x_data = data[:, :, :-1]
labels = data[:, 0, -1] # 첫 번째 프레임의 라벨만 사용 (시퀀스 내 라벨 일관성 가정)
print(f"x_data shape: {x_data.shape}")
print(f"labels shape: {labels.shape}")

Concatenated data shape: (4401, 30, 199)
x_data shape: (4401, 30, 198)
labels shape: (4401,)


3. 원-핫 인코딩

In [25]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
print(f"y_data shape (one-hot encoded): {y_data.shape}")

y_data shape (one-hot encoded): (4401, 12)


4. 학습/검증 데이터 분할

In [26]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2025)

print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}, y_val shape: {y_val.shape}")

x_train shape: (3960, 30, 198), y_train shape: (3960, 12)
x_val shape: (441, 30, 198), y_val shape: (441, 12)


5. 모델 구성

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

In [35]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(64, input_shape=x_train.shape[1:3]),
    Dropout(0.2), # LSTM 출력에 20% 드롭아웃 적용
    Dense(32, activation='relu'),
    Dropout(0.2), # Dense 출력에 20% 드롭아웃 적용
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 64)                67328     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_7 (Dense)             (None, 12)                396       
                                                                 
Total params: 69,804
Trainable params: 69,804
Non-trainable params: 0
_________________________________________________________________


6. 모델 학습

In [36]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# models 디렉토리가 없으면 생성
os.makedirs('models', exist_ok=True)

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=30,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

Epoch 1/30
123/124 [============================>.] - ETA: 0s - loss: 1.3601 - acc: 0.6367
Epoch 1: val_loss improved from inf to 0.37451, saving model to models\model.h5
124/124 [==============================] - 11s 39ms/step - loss: 1.3556 - acc: 0.6381 - val_loss: 0.3745 - val_acc: 0.9660 - lr: 0.0010
Epoch 2/30
120/124 [============================>.] - ETA: 0s - loss: 0.3388 - acc: 0.9378
Epoch 2: val_loss improved from 0.37451 to 0.08685, saving model to models\model.h5
124/124 [==============================] - 3s 22ms/step - loss: 0.3359 - acc: 0.9386 - val_loss: 0.0868 - val_acc: 0.9887 - lr: 0.0010
Epoch 3/30
123/124 [============================>.] - ETA: 0s - loss: 0.1504 - acc: 0.9710
Epoch 3: val_loss improved from 0.08685 to 0.02200, saving model to models\model.h5
124/124 [==============================] - 3s 28ms/step - loss: 0.1502 - acc: 0.9710 - val_loss: 0.0220 - val_acc: 1.0000 - lr: 0.0010
Epoch 4/30
124/124 [==============================] - ETA: 0s - loss: 0.0